# 2025 ASAP CRN Colloborative Meeting - CRN Cloud Training Workshop Notebook #2

## Overview
This notebook is part 2: **Data Composition to create a smaller (more wieldy) _sub_- dataset**

## Outline 

2. [`AnnData` Data Composition](#part-2-anndata-data-composition)
    * Load unfiltered dataset `AnnData` object
    * combine with _processed_ artifacts (`adata.obs`) 
    * Subset to a single brain region and Case-Contol group (from _dataset_ metadata)
        * annotate cell metadata with dataset metadata
        * save full gene expression `anndata` for subset with full metadata annotation (_dataset_ and _cell_-level)




## Notebooks
All of this content can be found in shorter more specific notebooks.  E.g. During the workshop you can follow along for Parts 1 & 2, but should start with Part 3 ([03_GOI_analysis.ipynb](./03_GOI_analysis.ipynb))

- This notebook [00_full_example_notebook.ipynb](./00_full_example_notebook.ipynb)
    - Part 1 - [01_basic_EDA.ipynb](./01_basic_EDA.ipynb)  
    - Part 2 - [02_dataset_composition.ipynb](./02_dataset_composition.ipynb)
    - Part 3 - [03_GOI_analysis.ipynb](./03_GOI_analysis.ipynb)
        - Part 3-R - [03-R_GOI_analysis.ipynb](./03-R_GOI_analysis.ipynb)
    - Part 4 - [04_DE_analysis.ipynb](./04_DE_analysis.ipynb)



------------------------------
>> NOTES
>>   need to develop an equivalent R version for part 1,2, and 4

## Imports

In [1]:
import numpy as np
import pandas as pd

# Use pathlib for file path manipulation
from pathlib import Path 


# matplotlib and seaborn a pythonic alternative to plotnine
try:
    import seaborn as sns
    import matplotlib.pyplot as plt
except ImportError as e:
    print("Error -> ", e)
    print("Installing seaborn or matplotlib")
    !pip install matplotlib seaborn
    import seaborn as sns
    import matplotlib.pyplot as plt


try:
    import scanpy as sc
except ImportError as e:
    print("Error -> ", e)
    print("Installing scanpy")
    !pip install scanpy
    import scanpy as sc

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

# Always show all columns in a Pandas DataFrame
pd.set_option('display.max_columns', None)

time: 252 μs (started: 2025-03-06 08:52:12 +00:00)


###  ASAP CRN data paths
First, let's build the paths to our data. 

In [2]:
# Workspace Path
HOME_PATH = Path.home()
WS_PATH =  HOME_PATH / 'workspace'
if not WS_PATH.exists():
    print(f"{WS_PATH} doesn't exist. We need to remount our resources")
    !wb resource mount    

time: 682 μs (started: 2025-03-06 08:52:12 +00:00)


We'll focus in on the datasets processed with our *PMDBS scRNAseq* workflow.  Specifically the _cohort_ dataset: `asap-cohort-pmdbs-sc-rnaseq`.  This dataset includes samples from 5 Contributing datasets which have been processesed and integrated.  The paths include the following parts.

- `workflow` designates the workflow which performs the aggregation and integration.  In this case the [*PMDBS scRNAseq* workflow](https://github.com/ASAP-CRN/pmdbs-sc-rnaseq-wf)
- `dataset_team` designates the contributing team for the dataset.  In this case _cohort_ designates that it is made from multiple individual contributed datasets.
- `source` designates the _source_ of the samples.  In this case Post-mortem derived Brain samples
- `dataset_type` designates the 
- `bucket_name` designates the datasets gcp bucket
- `dataset_name` designates the unique designation for each dataset or collection

In [3]:
DATASETS_PATH = WS_PATH / "01_PMDBS_scRNAseq_Datasets"
workflow = "pmdbs_sc_rnaseq"
dataset_team = "cohort"
dataset_source = "pmdbs"
dataset_type = "sc-rnaseq"
bucket_name = f"asap-curated-{dataset_team}-{dataset_source}-{dataset_type}"
dataset_name = f"asap-{dataset_team}-{dataset_source}-{dataset_type}"
dataset_path = DATASETS_PATH / bucket_name / workflow

time: 759 μs (started: 2025-03-06 08:52:15 +00:00)


#### Cohort Analysis Path

Now that we've defined the path to our cohort dataset, lets list the curated files for the `cohort_analysis`.   

In [4]:
cohort_analysis_path = dataset_path / "cohort_analysis"

time: 357 μs (started: 2025-03-06 08:52:16 +00:00)


#### Dataset Metadata Path

The dataset metadata can be found in the `release_resources`.  Note that the metadata are organized by the _short_ `dataset_name` rather than `bucket_name`.  



In [5]:
ds_metadata_path = WS_PATH / 'release_resources/asap-crn-cloud-release-resources' / dataset_name / "metadata"

!ls {ds_metadata_path} 

ASSAY_RNAseq.csv  CONDITION.csv  PMDBS.csv     SAMPLE.csv  SUBJECT.csv
CLINPATH.csv	  DATA.csv	 PROTOCOL.csv  STUDY.csv
time: 854 ms (started: 2025-03-06 08:52:30 +00:00)


#### Workshop/Notebook Artifact Path

This is the path to where the artifacts we've created in these notebooks can be found in case you want to skip ahead.  Particularly for the workshop.  In which case you can replace the `local_data_path` with `WORKSHOP_PATH`.

In [6]:
WORKSHOP_PATH = WS_PATH / "release_resources/asap-crn-cloud-release-resources/release-artifacts/2025_CRN_CM_Workshop_Resources_03122025" 

!ls {WORKSHOP_PATH}

data  notebooks
time: 658 ms (started: 2025-03-06 08:52:40 +00:00)


#### Local Data Path
Lets also define a path for copying our data files and exporting intermediate analysis artifacts to your workspace.  In this example we'll make a "workshop_files" in the "ws_files" which are persistent in our Verily Workbench Workspace. 

In [7]:
local_data_path = WS_PATH / "workshop_files"

if not local_data_path.exists():
    local_data_path.mkdir(parents=True)

time: 1.94 ms (started: 2025-03-06 08:52:42 +00:00)


--------------

# Part 2: `AnnData` Data Composition 



The full `asap-cohort` PMDBS snRNAseq dataset contains gene expression (~36k genes) measurments of 2.7m cells.   We will subset to a smaller dataset of 290k prefrontal cortex cells.   

We will make this `AnnData` artifact available as the resources of loading the full dataset and subsetting can be.

An alternate strategy is to load each individual dataset individually, which is less resource intensive.  However, we would like to leverage the shared latent space for visualizations. 


Steps here.

* Load unfiltered dataset `AnnData` object
* combine with _processed_ artifacts (`adata.obs`) 
* Subset to a "frontal_ctx" brain region and Case-Contol group (from _dataset_ metadata)
    * save full gene expression `anndata` for subset with full metadata annotation (_dataset_ and _cell_-level)



### Subset by sample's brain region 

In the following we'll want to do a psuedo bulk differential expression analysis and also a simple meta-analysis over the individual contributions within the pan-ASAP CRN _cohort_ datasets.    Note that we are choosing to subset from the full asap-cohort so we can leverage the common UMAP.

First we need to load the _dataset_ metadata from Part 1.


In [8]:
dataset_metadata_filen = local_data_path / "asap-cohort-dataset-metadata.csv"

df = pd.read_csv(dataset_metadata_filen)

time: 16.1 ms (started: 2025-03-06 08:54:02 +00:00)


And then we can make the mapings from sample IDs to the metadata values.

In [9]:
# define sample to match
br_mapper_full = dict(zip(df["sample"], df["brain_region"]))
br_mapper_simple = dict(zip(df["sample"], df["brain_region_simple"]))

# Parkinsons and control samples
condition_id_mapper = dict(zip(df["sample"], df["condition_id"]))
case_id_mapper = dict(zip(df["sample"], df["intervention_id"]))

time: 2.39 ms (started: 2025-03-06 08:54:17 +00:00)


In [11]:
adata_filename = local_data_path / f"asap-{dataset_team}.final_adata.h5ad"
adata = sc.read_h5ad(adata_filename, backed="r")

# add brain region to adata.obs
obs = adata.obs.copy()

obs["brain_region"] = obs["sample"].map(br_mapper_full)
obs["brain_region_simple"] = obs["sample"].map(br_mapper_simple)
obs["case_id"] = obs["sample"].map(case_id_mapper)

frontal_cells = obs["brain_region_simple"] == "frontal_ctx"

OSError: Unable to synchronously open file (truncated file: eof = 5634523136, sblock->base_addr = 0, stored_eof = 8513565110)

time: 123 ms (started: 2025-03-06 08:58:30 +00:00)


### Subset to "Case" or "Control" samples

Some of the contributed datasets include "Other" (e.g. 'prodromal PD') conditions besides "Ideopathic Parkinson's" and "Healthy Control".   We'll make a simplified dataset with just samples from Case (Parkinson's) and Control.  

In [ ]:
case_control_cells = ~(obs["case_id"] == "Other")  # exclude "Other"

In [ ]:
include = frontal_cells & case_control_cells


NOTE: We read the full 2.7m Cells X 36k genes from an h5ad file, NOT into memory.  Here we are copying the 291k cells into memory and closing the file.

In [ ]:
frontal_ad = adata[include].to_memory()
adata.file.close()  # close the original adata file

In [ ]:
# add in any other metadata we'd like to use later?
frontal_ad.obs["case_control"] = frontal_ad.obs["sample"].map(case_id_mapper)
frontal_ad.obs["condition_id"] = frontal_ad.obs["sample"].map(condition_id_mapper)
frontal_ad.obs["case_id"] = frontal_ad.obs["sample"].map(case_id_mapper)
frontal_ad.obs["brain_region"] = frontal_ad.obs["sample"].map(br_mapper_simple)
frontal_ad

Now we have a much smaller ~300k cells to work with, roughtly balanced btween Case and Control.

In [ ]:
frontal_ad.obs["case_id"].value_counts()

Export subset for merging with full gene expression to our `local_data_path`.

In [ ]:
frontal_samples_filename = (
    local_data_path / f"asap-{dataset_team}.frontal_ctx_case_control_samples.h5ad"
)
frontal_ad.write_h5ad(frontal_samples_filename)

Although we've subset to a much smaller dataset, we can see in our UMAP visualization that we have a wide ssampling of the overall gene expression latent space. 

In [ ]:
sc.pl.embedding(frontal_ad, basis="umap", color=["case_control"])

In [ ]:
del adata, frontal_ad

## Compose subset `AnnData` summary object 

### Copy dataset `anndata` to workspace 

Loading the dataset directly from the gcp bucket is inefficient.  Lets first copy it to our workspace files.

In [ ]:
full_adata_filename = (
    cohort_analysis_path / f"asap-{dataset_team}.merged_adata_object.h5ad"
)
l_full_adata_filename = (
    local_data_path / f"asap-{dataset_team}.merged_adata_object.h5ad"
)

In [ ]:
if not l_full_adata_filename.exists():
    !cp {full_adata_filename} {l_full_adata_filename}

### Load _full_ gene expression 
 
We'll construct an adata having the full (not limited to highly variable genes) gene expression, by combining the the subset adata (with all the embeddings and analyses) with the `asap-cohort.merged_adata_object.h5ad`.

In [ ]:
full_adata = sc.read_h5ad(l_full_adata_filename, backed="r")

frontal_samples_filename = (
    local_data_path / f"asap-{dataset_team}.frontal_ctx_case_control_samples.h5ad"
)
frontal_ad = sc.read_h5ad(frontal_samples_filename, backed="r")

var_ = full_adata.var.copy()
X = full_adata[frontal_ad.obs_names].X.copy()

full_adata.file.close()

Now we can combine the _full_ gene expression matrix wit ouyr frontal cortex subset, and save the resulting `AnnData`object.

In [ ]:
frontal_full_ad = sc.AnnData(
    X=X,
    obs=frontal_ad.obs,
    var=var_,
    uns=frontal_ad.uns,
    obsm=frontal_ad.obsm,
)

### Export full frontal cortex `AnnData` object for further analysis


In [ ]:
frontal_full_samples_filename = (
    local_data_path / f"asap-{dataset_team}.full_frontal_ctx_case_control_samples.h5ad"
)
frontal_full_ad.write_h5ad(frontal_full_samples_filename)

-----------------
# Next:   
Continue with:
- Part 3 - Gene of interest analysis (__Workshop Activity__) [03_GOI_analysis.ipynb](./03_GOI_analysis.ipynb)


--------------------
# Provenance
Generate information about this notebook environment and the packages installed.

In [ ]:
!date

pip installed packages:

In [ ]:
!pip freeze

JupyterLab extensions:

In [ ]:
!jupyter labextension list

Number of cores:

In [ ]:
!grep ^processor /proc/cpuinfo | wc -l

Memory:

In [ ]:
!grep "^MemTotal:" /proc/meminfo